In [2]:
import os
import audiosegment
import numpy as np

In [3]:
sample_rate = 22050
data_directory = "/home/alexm/Datasets/Trump/"
db_values = []

In [ ]:
for file in sorted(os.listdir(data_directory)):
    filename = data_directory + os.fsdecode(file)
    print("Current Filename: %s" % filename)

    if filename.endswith(".wav"):
        try:
            audio = audiosegment.from_file(filename).resample(sample_rate_Hz=sample_rate)
            wav = audio.to_numpy_array()

            if len(wav.shape) == 2:
                wav = wav.T.flatten()

            if wav.dtype == np.int16:
                wav = wav / 32768.0
            elif wav.dtype == np.int32:
                wav = wav / 2147483648.0
            elif wav.dtype == np.uint8:
                wav = (wav - 128) / 128.0

            wav = wav.astype(np.float32).tolist()
            db_values.append(wav)
        except:
            print("Failed to extract sound data/statistics from %s" % filename)
        continue
    else:
        continue

Current Filename: /home/alexm/Datasets/Trump/prompts.gui
Current Filename: /home/alexm/Datasets/Trump/trump1000000000.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000000.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000001.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000002.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000003.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000004.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000005.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000006.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000007.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000008.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000009.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000001.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000010.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000011.wav
Current Filename: /ho

Current Filename: /home/alexm/Datasets/Trump/trump10000000148.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000149.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000015.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000150.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000151.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000152.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000153.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000154.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000155.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000156.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000157.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000158.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000159.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000016.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000160.wav
Current File

Current Filename: /home/alexm/Datasets/Trump/trump10000000296.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000297.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000298.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000299.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000030.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000300.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000301.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000302.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000303.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000304.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000305.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000306.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000307.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000308.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000309.wav
Current Fil

Current Filename: /home/alexm/Datasets/Trump/trump10000000437.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000438.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000439.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000044.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000440.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000441.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000442.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000443.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000444.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000445.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000446.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000447.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000448.wav
Current Filename: /home/alexm/Datasets/Trump/trump10000000449.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000045.wav
Current File

Current Filename: /home/alexm/Datasets/Trump/trump1000000091.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000092.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000093.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000094.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000095.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000096.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000097.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000098.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000099.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000100.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000101.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000102.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000103.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000104.wav
Current Filename: /home/alexm/Datasets/Trump/trump1000000105.wav
Current Filename: /home/a

...

Current Filename: /home/alexm/Datasets/Trump/trump11000000599.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000600.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000601.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000602.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000603.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000604.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000605.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000606.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000607.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000608.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000609.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000610.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000611.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000612.wav
Current Filename: /home/alexm/Datasets/Trump/trump11000000613.wav
Current Fi

Current Filename: /home/alexm/Datasets/Trump/trump12000000096.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000097.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000098.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000099.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000100.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000101.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000102.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000103.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000104.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000105.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000106.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000107.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000108.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000109.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000110.wav
Current Fi

Current Filename: /home/alexm/Datasets/Trump/trump12000000221.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000222.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000223.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000224.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000225.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000226.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000227.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000228.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000229.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000230.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000231.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000232.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000233.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000234.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000235.wav
Current Fi

Current Filename: /home/alexm/Datasets/Trump/trump12000000346.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000347.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000348.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000349.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000350.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000351.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000352.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000353.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000354.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000355.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000356.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000357.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000358.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000359.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000360.wav
Current Fi

Current Filename: /home/alexm/Datasets/Trump/trump12000000495.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000496.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000497.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000498.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000499.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000500.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000501.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000502.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000503.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000504.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000505.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000506.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000507.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000508.wav
Current Filename: /home/alexm/Datasets/Trump/trump12000000509.wav
Current Fi

In [ ]:
db_values = np.array(db_values)
db_values.flatten()
print(db_values.size)

In [ ]:
print('Min: %f; Max: %f; Mean: %f; Median: %f' % (np.min(db_values), np.max(db_values), np.mean(db_values), np.median(db_values)))